1. train and test diabetes
2. Remove outliers using Z score. Usual guideline is to remove anything that has Z score > 3 formula or Z score < -3
3. Apply scaling
4. Build a classification model using support vector machine. Use standalone model as well as Bagging model and check if you see any difference in the performance.
5. Use decision tree classifier. Use standalone model as well as Bagging and check if you notice any difference in performance
6. Comparing performance of svm and decision tree classifier figure out where it makes most sense to use bagging and why. Use internet to figure out in what conditions bagging works the best.

In [9]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("diabetes.csv")
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [18]:
# Remove outliers using Z score
from scipy.stats import zscore
zscores = np.abs(zscore(df))
threshold = 3
df_filt = df[(zscores < threshold)].dropna()

x = df.drop(columns = ["Outcome"])
y = df.Outcome

# apply scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_scaled,y,test_size = 0.2)

# Build a classification model using support vector machine
from sklearn.svm import SVC
svc = SVC()
svc.fit(x_train,y_train)
svc_score = svc.score(x_test,y_test)
print(f"The test score of SVC model is {round(svc_score,4)}.")

# Bagging -- using SVM model
from sklearn.ensemble import BaggingClassifier
bag_model = BaggingClassifier(base_estimator = SVC(),
                              n_estimators = 5,max_samples =  0.8,
                              oob_score = True,random_state = 0)
bag_model.fit(x_train,y_train)
oob_score = bag_model.oob_score_
bag_score = bag_model.score(x_test,y_test)
print(f"The out-of-bag score is {round(oob_score,4)}.")
print(f"The test score is {round(bag_score,4)}.")
print()

# Build a classification model using support vector machine
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(x_train,y_train)
dt_score = dt.score(x_test,y_test)
print(f"The test score of Decision Tree Classifier model is {round(dt_score,4)}.")

# Bagging -- using Decision Tree Classifier model
bag_model_0 = BaggingClassifier(base_estimator = DecisionTreeClassifier(),
                              n_estimators = 5,max_samples =  0.8,
                              oob_score = True,random_state = 0)
bag_model_0.fit(x_train,y_train)
oob_score_0 = bag_model_0.oob_score_
bag_score_0 = bag_model_0.score(x_test,y_test)
print(f"The out-of-bag score is {round(oob_score_0,4)}.")
print(f"The test score is {round(bag_score_0,4)}.")

The test score of SVC model is 0.7597.
The out-of-bag score is 0.7524.
The test score is 0.7727.

The test score of Decision Tree Classifier model is 0.7468.
The out-of-bag score is 0.7231.
The test score is 0.7403.


- Comparing performance of svm and decision tree classifier figure out where it makes most sense to use bagging and why.

In my opinion, bagging model performs better on SVM